In [1]:
import pandas as pd
import nltk
df1 = pd.read_csv('path',usecols=['title','content'],encoding='ISO-8859-1')
df2 = pd.read_csv('path',usecols=['title','content'],encoding='ISO-8859-1')
df3 = pd.read_csv('path',usecols=['title','content'],encoding='ISO-8859-1')

In [2]:
 frames = [df1, df2, df3]

In [3]:
df=pd.concat(frames)

In [4]:
df.count

<bound method DataFrame.count of                                                    title  \
0      House Republicans Fret About Winning Their Hea...   
1      Rift Between Officers and Residents as Killing...   
2      Tyrus Wong, âBambiâ Artist Thwarted by Rac...   
3      Among Deaths in 2016, a Heavy Toll in Pop Musi...   
4      Kim Jong-un Says North Korea Is Preparing to T...   
5      Sick With a Cold, Queen Elizabeth Misses New Y...   
6      Taiwanâs President Accuses China of Renewed ...   
7      After âThe Biggest Loser,â Their Bodies Fo...   
8      First, a Mixtape. Then a Romance. - The New Yo...   
9      Calling on Angels While Enduring the Trials of...   
10     Weak Federal Powers Could Limit Trumpâs Clim...   
11     Can Carbon Capture Technology Prosper Under Tr...   
12     Mar-a-Lago, the Future Winter White House and ...   
13     How to form healthy habits in your 20s - The N...   
14     Turning Your Vacation Photos Into Works of Art...   
15     

In [5]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [6]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [7]:
def text_tokenizer(text, stops):
    import re
    # 1. Eliminate markup
    text = re.sub("\<[^<>]*\>"," ", text.lower())
    # 2. Separate review strings from other types of sequences
    text = re.sub("([a-z]+|[^a-z]+)",r"\1 ", text)
    # 3. delete anything not a review, digit, $, or %
    text = re.sub("[^a-z ]","", text)
    # 4. Clean up multiple white spaces
    text = re.sub("\s+"," ", text)
    text = re.sub("^\s+","", text)
    text = re.sub("\s+$"," ", text)
    # 5. Split sentences to words
    words = text.split()
    # 6. Keep words in the `keeps`
    words = [w for w in words if w not in stops]
    # 7. Return a list of words
    return words

# Define a function to split a review into parsed sentences
def text_to_sentences(text, keeps):
    import nltk
    # Use the NLTK tokenizer to split the paragraph into sentences
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(text.strip().encode().decode('utf-8'))

    # Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(text_tokenizer(raw_sentence,keeps))
    return sentences

In [8]:
# read-in stopwords# read- 
with open('C:/Users/Fatima/AppData/Roaming/nltk_data/corpora/stopwords/english','r+') as fobj:
    stops = fobj.read().split()
fobj.closed 

True

In [9]:
# construct a collection of sentences for word-embedding learning.# const 
import time

# Initialize an empty list of sentences
sentences = [] 

print('Parsing sentences')
tic = time.clock()
for text in df['content'].astype(str):
    sentences += text_to_sentences(text,stops)
toc = time.clock()
toc - tic

Parsing sentences


1556.5484543336872

In [10]:
# Word2Vec Training
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
from gensim.models import word2vec
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

model.init_sims(replace=True)


C:\Users\Fatima\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [11]:
word_vectorsword_ve  = model.wv.syn0


C:\Users\Fatima\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [12]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #bag of cenroids
    num_centroids = max( word_centroid_map.values() ) + 1
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    return bag_of_centroids

In [13]:
n_components=60
from sklearn.mixture import GMM
gmm = GMM(n_components)
idx = gmm.fit_predict(word_vectorsword_ve)

word_centroid_map = dict(zip( model.wv.index2word, idx ))
for cluster in range(0,n_components):
     print ("\nCluster %d" % cluster)
     words = []
     for i in range(0,len(word_centroid_map.values())):
              if( list(word_centroid_map.values())[i] == cluster ):
                     words.append(list(word_centroid_map.keys())[i])
     print(words)



C:\Users\Fatima\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Fatima\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Fatima\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Fatima\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning:

C:\Users\Fatima\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Fatima\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Fatima\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Fatima\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Functi


Cluster 0
['people', 'many', 'country', 'women', 'black', 'among', 'americans', 'men', 'often', 'social', 'others', 'across', 'young', 'community', 'live', 'groups', 'lives', 'large', 'especially', 'share', 'middle', 'living', 'families', 'particularly', 'immigrants', 'popular', 'areas', 'class', 'citizens', 'cities', 'poor', 'residents', 'population', 'ones', 'female', 'muslims', 'mostly', 'largely', 'communities', 'places', 'increasingly', 'gay', 'gender', 'racial', 'activists', 'african', 'older', 'male', 'minority', 'generally', 'rich', 'color', 'generation', 'vast', 'younger', 'typically', 'tend', 'poverty', 'native', 'heavily', 'diversity', 'centers', 'urban', 'adults', 'wide', 'elsewhere', 'vulnerable', 'rural', 'immigrant', 'grown', 'variety', 'youth', 'wealthy', 'lgbt', 'asian', 'similarly', 'handful', 'hispanic', 'diverse', 'whites', 'minorities', 'latino', 'participants', 'partly', 'neighborhoods', 'ordinary', 'historically', 'generations', 'mainly', 'primarily', 'ethnic', 

['past', 'recent', 'seen', 'face', 'despite', 'latest', 'result', 'biggest', 'continued', 'due', 'cause', 'growing', 'huge', 'throughout', 'deep', 'worst', 'massive', 'continues', 'caused', 'facing', 'spread', 'faced', 'faces', 'amid', 'ongoing', 'wake', 'suffered', 'slow', 'struggling', 'fresh', 'unprecedented', 'experienced', 'ending', 'apparent', 'driven', 'dealing', 'widespread', 'dramatic', 'divided', 'begun', 'surrounding', 'prompted', 'intense', 'exposed', 'marked', 'sharp', 'enormous', 'struggled', 'causing', 'constant', 'deeper', 'resulted', 'aftermath', 'suffer', 'desperate', 'emerging', 'devastating', 'sudden', 'frequent', 'sparked', 'reminder', 'dominated', 'unexpected', 'damaged', 'painful', 'subsequent', 'occur', 'string', 'fierce', 'inevitable', 'renewed', 'stunning', 'stark', 'sustained', 'spreading', 'resulting', 'troubled', 'tense', 'tragic', 'avoided', 'bitter', 'profound', 'generated', 'facebooksharetweet', 'fueled', 'witnessed', 'triggered', 'endless', 'threatens',

['comes', 'makes', 'whose', 'shows', 'takes', 'created', 'works', 'giving', 'appears', 'writes', 'suggests', 'becoming', 'gives', 'shown', 'becomes', 'contributed', 'greatest', 'unlike', 'offers', 'stands', 'explains', 'sees', 'represents', 'leads', 'uses', 'adds', 'unique', 'leaves', 'provides', 'inspired', 'puts', 'holds', 'begins', 'drawn', 'finds', 'follows', 'reflect', 'proved', 'serves', 'keeps', 'brings', 'argues', 'describes', 'speaks', 'sets', 'recognized', 'defined', 'proven', 'versus', 'raises', 'meets', 'exists', 'competing', 'creates', 'demonstrated', 'involves', 'depends', 'reflects', 'engaging', 'devoted', 'achieved', 'reveals', 'reflected', 'vs', 'besides', 'opens', 'contains', 'carries', 'considers', 'sends', 'depend', 'reserved', 'agrees', 'insists', 'gaining', 'drives', 'poses', 'highlights', 'presents', 'existed', 'focuses', 'influenced', 'affects', 'destroying', 'refers', 'admits', 'shaped', 'displayed', 'controlling', 'transformed', 'lacks', 'fits', 'evolved', 'ab

...

['reporting', 'editing', 'tokyo', 'simon', 'photographer', 'designer', 'seoul', 'chef', 'dubai', 'processor', 'wang', 'sydney', 'architect', 'slideshow', 'bernard', 'getty', 'holland', 'li', 'vantiv', 'worldpay', 'von', 'adler', 'hay', 'pioneer', 'chen', 'specializes', 'frankfurt', 'yang', 'bengaluru', 'cooney', 'renowned', 'dznussbaum', 'wong', 'qamishli', 'freelance', 'lin', 'oatis', 'oslo', 'chang', 'dalgleish', 'orr', 'wu', 'curator', 'liu', 'zhang', 'zieminski', 'shepherd', 'shumaker', 'ho', 'liquified', 'angus', 'lu', 'elliman', 'davies', 'jonas', 'rigby', 'gregorio', 'kang', 'alistair', 'osterman', 'nguyen', 'lucia', 'nv', 'dublin', 'mccool', 'roberta', 'flipboard', 'milliken', 'grebler', 'profiled', 'kumar', 'hans', 'realtor', 'trott', 'chung', 'colvin', 'edison', 'edinburgh', 'bosch', 'kingston', 'anthropologist', 'andreas', 'astronomer', 'mona', 'doina', 'welsh', 'chiacu', 'macdonald', 'giles', 'mazzilli', 'yi', 'chizu', 'nomiyama', 'fraser', 'sharma', 'yu', 'graff', 'chi', '

['john', 'david', 'ms', 'michael', 'james', 'george', 'j', 'mark', 'named', 'chris', 'robert', 'tom', 'richard', 'steve', 'ben', 'thomas', 'peter', 'jr', 'andrew', 'stephen', 'eric', 'jim', 'bob', 'veteran', 'william', 'charles', 'tim', 'anthony', 'daniel', 'matt', 'jack', 'jason', 'kevin', 'sarah', 'brian', 'mary', 'elizabeth', 'frank', 'michelle', 'dan', 'adam', 'jonathan', 'alex', 'matthew', 'joseph', 'josh', 'taylor', 'harry', 'susan', 'patrick', 'charlie', 'gary', 'howard', 'nick', 'tony', 'steven', 'jennifer', 'jon', 'jimmy', 'alexander', 'henry', 'justin', 'aaron', 'nancy', 'sam', 'roger', 'greg', 'ed', 'lisa', 'neil', 'christopher', 'larry', 'jay', 'recalls', 'rob', 'jane', 'russell', 'ann', 'jeffrey', 'angela', 'bruce', 'keith', 'benjamin', 'jerry', 'franklin', 'edward', 'amy', 'dave', 'joel', 'rachel', 'phil', 'terry', 'ian', 'alan', 'max', 'douglas', 'philip', 'ken', 'andy', 'courtesy', 'jeremy', 'barbara', 'carl', 'pastor', 'marc', 'ray', 'kate', 'barry', 'anne', 'laura', '

['twitter', 'facebook', 'data', 'online', 'phone', 'network', 'apple', 'internet', 'site', 'com', 'website', 'google', 'users', 'images', 'via', 'search', 'check', 'platform', 'ads', 'ad', 'content', 'computer', 'messages', 'digital', 'videos', 'app', 'software', 'sites', 'box', 'devices', 'feature', 'updates', 'device', 'mobile', 'screen', 'networks', 'iphone', 'cable', 'tools', 'user', 'instagram', 'youtube', 'items', 'web', 'tool', 'posts', 'archive', 'phones', 'feed', 'cameras', 'communication', 'computers', 'click', 'microsoft', 'mail', 'electronic', 'snap', 'x', 'alert', 'audio', 'zuckerberg', 'viral', 'stream', 'telephone', 'apps', 'switch', 'streaming', 'virtual', 'tracking', 'posting', 'subscribe', 'websites', 'smartphone', 'snapchat', 'platforms', 'random', 'database', 'channels', 'versions', 'messaging', 'android', 'hashtag', 'mon', 'echo', 'automatically', 'cellphone', 'mode', 'developers', 'maps', 'org', 'tips', 'pok', 'porn', 'button', 'lets', 'accessible', 'medium', 'fit

['years', 'percent', 'since', 'less', 'least', 'number', 'high', 'almost', 'nearly', 'half', 'record', 'rate', 'single', 'higher', 'growth', 'short', 'per', 'increase', 'fall', 'numbers', 'average', 'rise', 'total', 'low', 'largest', 'previous', 'lower', 'period', 'rates', 'prices', 'compared', 'performance', 'showing', 'levels', 'track', 'increased', 'decade', 'rose', 'markets', 'demand', 'overall', 'quarter', 'larger', 'drop', 'highest', 'range', 'fed', 'roughly', 'analysis', 'thanks', 'rising', 'analysts', 'survey', 'twice', 'relatively', 'size', 'fewer', 'estimated', 'sector', 'smaller', 'decline', 'percentage', 'dollar', 'count', 'bottom', 'net', 'double', 'scale', 'slightly', 'expectations', 'gap', 'inflation', 'significantly', 'pace', 'gains', 'trend', 'estimates', 'employment', 'reserve', 'ten', 'wages', 'combined', 'increases', 'zero', 'ratings', 'reduced', 'losses', 'earnings', 'predicted', 'index', 'stocks', 'degrees', 'recovery', 'turnout', 'unemployment', 'worldwide', 'sur

['state', 'states', 'government', 'public', 'law', 'court', 'administration', 'federal', 'security', 'officials', 'case', 'department', 'office', 'statement', 'already', 'members', 'general', 'order', 'justice', 'congress', 'executive', 'decision', 'process', 'legal', 'service', 'agency', 'attorney', 'judge', 'action', 'defense', 'cases', 'response', 'special', 'rules', 'staff', 'civil', 'list', 'ban', 'board', 'release', 'letter', 'immediately', 'employees', 'agreement', 'review', 'safety', 'additional', 'refugees', 'hearing', 'test', 'travel', 'actions', 'previously', 'legislation', 'eventually', 'agencies', 'lawmakers', 'independent', 'status', 'request', 'individuals', 'lawsuit', 'practice', 'upon', 'authority', 'commission', 'sanctions', 'emergency', 'lawyers', 'agents', 'ruling', 'refused', 'measure', 'regulations', 'proposal', 'complete', 'instance', 'specifically', 'orders', 'initial', 'warning', 'acting', 'basis', 'approval', 'courts', 'conduct', 'initially', 'unclear', 'suit'

In [61]:
import time
def classify(texts):
    sentences = [] 
    print('Parsing sentences')
    tic = time.clock()
    for text in texts:
        sentences += text_to_sentences(text,stops)
    toc = time.clock()
    m1=word_vectorsword_ve.
    idx = gmm.fit_predict(m1)
    cluster=idx.tolist
    print(cluster)

In [56]:
def classify(texts):
    m=model.
    idx = gmm.fit_predict(m)
    cluster=idx.tolist
    print(cluster)